# Getting Started with Agentune Simulate - Quick Start

This notebook provides a streamlined introduction to the Agentune Simulate library. You'll learn to:

- Load conversation data and extract outcomes
- Set up RAG-based simulation with in-memory vector store
- Run simulations and analyze results

**Note**: This is a simplified version. For persistent storage and production features, see `persistent_storage_example.ipynb`.

## Setup and Imports

In [1]:
import os
import getpass
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Outcomes
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder
from utils import setup_logging_and_asyncio, load_data_with_outcomes

## API Key Configuration

In [2]:
# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ API key configured")

✓ API key configured


## Environment Setup

In [3]:
# Configure logging and asyncio for Jupyter
setup_logging_and_asyncio()

✓ Logging configured
✓ Asyncio event loop configured for Jupyter


## Load Data and Extract Outcomes

In [4]:
# Load conversations and extract outcomes in one step
conversations, outcomes_tuple = load_data_with_outcomes("data/sample_conversations.csv")
outcomes = Outcomes(outcomes=outcomes_tuple)

Loading conversations from data/sample_conversations.csv...
✓ Loaded 50 conversations
✓ Sample conversation has 16 messages
✓ Extracted 2 unique outcomes
  - unresolved: Issue was not resolved
  - resolved: Issue was successfully resolved


## Create Models and Vector Store

In [5]:
# Setup models
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create in-memory vector store
documents = conversations_to_langchain_documents(conversations)
vector_store = InMemoryVectorStore.from_documents(documents, embeddings)

print(f"✓ Created vector store with {len(documents)} documents")

✓ Created vector store with 876 documents


## Run Simulation

In [6]:
# Build and run simulation session
session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=10
).build()

# Run simulation with first 5 conversations as starting points
base_conversations = conversations[:5]
result = await session.run_simulation(real_conversations=base_conversations)

print("✓ Simulation completed!")

2025-07-24 12:34:03,254 - Starting intent extraction on 5 conversations
2025-07-24 12:34:07,589 - Finished extracting original intents; generated 5 scenarios
2025-07-24 12:34:07,590 - Starting conversation simulations (self.max_concurrent_conversations=20)
2025-07-24 12:34:12,594 - Progress: 0/5 scenarios completed
2025-07-24 12:34:42,600 - Progress: 1/5 scenarios completed
2025-07-24 12:34:52,603 - Progress: 2/5 scenarios completed
2025-07-24 12:34:57,605 - Progress: 4/5 scenarios completed
2025-07-24 12:35:02,608 - Progress: 5/5 scenarios completed
2025-07-24 12:35:02,610 - Finished simulating conversations; simulated 5 conversations, with 0 failures
2025-07-24 12:35:02,611 - Starting analysis of simulation results
2025-07-24 12:35:08,353 - Finished analyzing results


✓ Simulation completed!


## Analyze Results

In [7]:
print(result.generate_summary())

SIMULATION RESULTS
Session name: Simulation Session
Original conversations: 5
Simulated conversations: 5

Average messages per conversation:
  Original: 15.6
  Simulated: 9.4

Outcome distribution comparison:
Outcome              Original        Simulated      
--------------------------------------------------
resolved               1 (20.0%)     0 ( 0.0%)
unresolved             4 (80.0%)     5 (100.0%)

Sample conversation (10 messages):
  1. agent: Thank you for calling PC Support, how can I help you today?
  2. customer: I'm having issues with my internet connection; it keeps dropping intermittently.
  3. agent: I'm sorry to hear that you're having connection issues. Let's start by checking if your internet con...
  4. customer: I'll run a speed test now and let you know the results.
  ... and 6 more messages


## Next Steps

Now that you've completed a basic simulation:

  1. **Use your own data**: Load your own conversations as a list of `Conversation`
  objects and use them to set up the simulation.
  2. **Production features**: See `persistent_storage_example.ipynb` for persistent
  storage with Chroma and advanced configurations suitable for larger datasets and
  production use.
  3. **Explore advanced features**: Check out the full documentation for more options
  like caching of LLM responses, LLM failures handling, and more.

  ### Resources:
  - [Full documentation](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/README.md)
  - [Complete examples](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples)
  - [Persistent storage example](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples/persistent_storage_example.ipynb)
  - [Streamlit web interface](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/streamlit/README.md)